This piece of code deals with df_stats, which contains information on wages, and the distribution of workers across occupations.

In [ ]:
function weightedmean(var,wgt)
    numer = 0
    denom = 0
    for i in 1:length(var) 
        numer += wgt[i] * var[i]
        denom += wgt[i]
    end
    return(numer/denom)
end

function def_mom14(data,  N::Hyper)
    mom1 = fill(NaN, N.J)
    mom4 = fill(NaN, N.J)
    for row in 1:length(data[:,1])
        j = data.occ1[row]
        if data.emp1[row]==1
            mom1[j] = data.w[row]===missing ? NaN : data.w[row]
            mom4[j] = data.N[row]===missing ? NaN : data.N[row]
        end    
    end
    totpop = sum(data.N)
    return(mom1, mom4 ./ totpop )
end

This code deals with df_rates, which contains the transition probabilities between each state.
We build five $J\times J$ matrices:
1. pUU to transit from unemployment to unemployment (diagonal matrix),
2. pUE to transit from unemployment to employment,
3. pEU to transit from employment to unemployment (diagonal),
4. pEEnew to change employer according to question in CPS and same occupation
5. pEEsame to keep the same employer (diagonal)
We can generate pEE=pEEsame+pEEnew so that
$$\left(\begin{array}
 pUU & pUE \\
pEU & pEE
\end{array}\right)
$$
defines the transition probabilities. The terms on the same rows sum up to 1.
We apply Shimer (2005) to obtain the job separation rates and the job-finding rates.

With n_transit, we count the number of observations

In [ ]:
function share_transit(data,  N::Hyper)
    shUU = fill(0., N.J, N.J ) 
    shUE = fill(0., N.J,N.J)
    shEU = fill(0., N.J,N.J)
    shEEnew = fill(0., N.J,N.J)
    shEEsame = fill(0., N.J,N.J)
    for row in 1:length(data[:,1])
        s0 = data.state0[row]
        s1 = data.state1[row]
        if (s0[1]=='U')
            j0 = parse(Int,s0[2:end]) 
            if (s1[1]=='U') ## U2U
                j1 = parse(Int,s1[2:end]) 
                shUU[j0,j1] = data.rate[row]===missing ? NaN : data.rate[row] 
            elseif (s1[1:2]=="En") ## U2Enew, U2Esame=0 
                j1 = parse(Int,s1[5:end]) 
                shUE[j0,j1] = data.rate[row]===missing ? NaN : data.rate[row] 
            end
        elseif (s0[1]=='E')  
            j0 = parse(Int,s0[2:end]) 
            if (s1[1]=='U') ## E2U
                j1 = parse(Int,s1[2:end]) 
                shEU[j0,j1] = data.rate[row]===missing ? NaN : data.rate[row] 
            elseif (s1[1:2]=="En") ## E2Enew, U2Esame=0 
                j1 = parse(Int,s1[5:end]) 
                shEEnew[j0,j1] = data.rate[row]===missing ? NaN : data.rate[row] 
            elseif (s1[1:2]=="Es") ## E2Enew, U2Esame=0 
                j1 = parse(Int,s1[6:end]) 
                shEEsame[j0,j1] = data.rate[row]===missing ? NaN : data.rate[row] 
            end
        end
    end
    return(shUU,shUE,shEU,shEEnew,shEEsame)
end

function n_transit(data, N::Hyper)
    nUU = fill(0., N.J, N.J ) 
    nUE = fill(0., N.J,N.J)
    nEU = fill(0., N.J,N.J)
    nEEnew = fill(0., N.J,N.J)
    nEEsame = fill(0., N.J,N.J)
    for row in 1:length(data[:,1])
        s0 = data.state0[row]
        s1 = data.state1[row]
        if (s0[1]=='U')
            j0 = parse(Int,s0[2:end]) 
            if (s1[1]=='U') ## U2U
                j1 = parse(Int,s1[2:end]) 
                nUU[j0,j1] = data.N[row]===missing ? NaN : data.N[row] 
            elseif (s1[1:2]=="En") ## U2Enew, U2Esame=0 
                j1 = parse(Int,s1[5:end]) 
                nUE[j0,j1] = data.N[row]===missing ? NaN : data.N[row] 
            end
        elseif (s0[1]=='E')  
            j0 = parse(Int,s0[2:end]) 
            if (s1[1]=='U') ## E2U
                j1 = parse(Int,s1[2:end]) 
                nEU[j0,j1] = data.N[row]===missing ? NaN : data.N[row] 
            elseif (s1[1:2]=="En") ## E2Enew, U2Esame=0 
                j1 = parse(Int,s1[5:end]) 
                nEEnew[j0,j1] = data.N[row]===missing ? NaN : data.N[row] 
            elseif (s1[1:2]=="Es") ## E2Enew, U2Esame=0 
                j1 = parse(Int,s1[6:end]) 
                nEEsame[j0,j1] = data.N[row]===missing ? NaN : data.N[row] 
            end
        end
    end
    return(nUU,nUE,nEU,nEEnew,nEEsame)
end


function def_mom23(shUE, shEE, N::Hyper)
    ## Shimer (2005)
    pUE = shUE
    pEE = fill(0., N.J, N.J)
    pEU = fill(NaN, N.J)
    
    shEU = [1-sum(shEE[j0,:]) for j0 in 1:N.J] ##alternatively, we could use shEU
    for j0 in 1:N.J
        pEU[j0] = shEU[j0] / (1-0.5 * sum(pUE[j0,:]))
    end
    
    for j0 in 1:N.J, j1 in 1:N.J
        if (j0 != j1)
            pEE[j0,j1] = shEE[j0,j1] - pEU[j0] * 0.5 * pUE[j0,j1]
        end
    end
    
    mom2 = pUE
    mom3 = pEE
    jdr = pEU
    return(mom2,mom3,jdr)
end